# Data Imports

In [1]:
# let's start with the data and see how it goes
import os
import pandas as pd
HOME = os.getcwd()
train_csv = os.path.join(HOME, 'data', 'train.csv')
test_csv = os.path.join(HOME, 'data', 'test.csv')

df_train = pd.read_csv(train_csv)
df_test = pd.read_csv(test_csv)
# set the columns names to lower case 

df_train.columns = [c.lower() for c in df_train.columns]
df_test.columns = [c.lower() for c in df_test.columns]

# remove unnecessary columns
df_train.drop(columns=['helpfulness', 'score'], inplace=True)
df_test.drop(columns=['helpfulness', 'score'], inplace=True)

In [2]:
# add a small piece of code to call the pytorch_modular code
from pathlib import Path
import sys

current = HOME
while 'src' not in os.listdir(current):
    current = Path(current).parent

sys.path.append(str(current))
sys.path.append(os.path.join(current, 'src'))

In [3]:
df_train.head()

,title,text,category
0,Golden Valley Natural Buffalo Jerky,The description and photo on this product need...,grocery gourmet food
1,Westing Game,This was a great book!!!! It is well thought t...,toys games
2,Westing Game,"I am a first year teacher, teaching 5th grade....",toys games
3,Westing Game,I got the book at my bookfair at school lookin...,toys games
4,I SPY A is For Jigsaw Puzzle 63pc,Hi! I'm Martine Redman and I created this puzz...,toys games


In [4]:
df_test.head()

,id,title,text
0,0,PetSafe Staywell Pet Door with Clear Hard Flap,We've only had it installed about 2 weeks. So ...
1,1,"Kaytee Timothy Cubes, 1-Pound",My bunny had a hard time eating this because t...
2,2,Body Back Buddy,would never in a million years have guessed th...
3,3,SnackMasters California Style Turkey Jerky,"Being the jerky fanatic I am, snackmasters han..."
4,4,Premier Busy Buddy Tug-a-Jug Treat Dispensing ...,Wondered how quick my dog would catch on to th...


In [5]:
import nltk 
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

try:
    STOP_WORDS = list(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    STOP_WORDS = list(stopwords.words('english'))

In [6]:
# preprocessing functions
import re
from typing import List

def to_lower(text: str) -> str:
    return text.lower()

def no_extra_spaces(text: str) -> str:
    return re.sub('\s+', ' ', text)

def no_extra_chars(text: str) -> str:
    return re.sub(r'[^a-zA-Z\s,!.;:-]+', ' ', text) 

text = 'aaa5531--==-||"z2::,.a'

def remove_stop_words(text: str,
                      tokenizer: TweetTokenizer = None) -> str:
    text = to_lower(text)    
    tokenizer = TweetTokenizer() if tokenizer is None else tokenizer
    tokens = tokenizer.tokenize(text)
    # if the remove_stop_words argument is set to True, then filter stop words
    tokens = [t.strip() for t in tokens if t not in STOP_WORDS] 
    return " ".join(tokens)

def process(text: str) -> str:
    # first lower, remove extrac chracters
    text1 = to_lower(no_extra_chars(text))
    # remove redundant words
    text2 = remove_stop_words(text1)
    # remove extra spaces
    return no_extra_spaces(text2)

import random
random.seed(69)
example = df_train['text'][int(random.random() * len(df_train))]
print(example)
print(process(example))

# # drop the 'text' column as only the title will be used for classification
df_train.drop(columns=['text'], inplace=True)
df_test.drop(columns=['text'], inplace=True)

# 16 rows have missing values in the 'title' column, remove them
df_train.dropna(inplace=True)

See the title of this review. Fortunately, I am a packrat, and kept a bunch of hole repair kits from various blow up things that we have gone through over the years. Does not come with a hole repair kit though, just to warn you. Anyway, it is back in black and bouncing our 3 year old all over the place. Indoor only, I would say. Very highly recommended, in spite of a hole within a week of use. Hope that this is the first and last one... probably not.
see title review . fortunately , packrat , kept bunch hole repair kits various blow things gone years . come hole repair kit though , warn . anyway , back black bouncing year old place . indoor , would say . highly recommended , spite hole within week use . hope first last one ... probably .


In [7]:
print(df_train.isna().sum())
print("#" * 100)
print(df_test.isna().sum())

title       0
category    0
dtype: int64
####################################################################################################
id       0
title    5
dtype: int64


In [8]:
import numpy as np

cat2idx = {
    'toys games': 0,
    'health personal care': 1,
    'beauty': 2,
    'baby products': 3,
    'pet supplies': 4,
    'grocery gourmet food': 5,
}

idx2cat = {
    0:'toys games',
    1:'health personal care',
    2:'beauty',
    3:'baby products',
    4:'pet supplies',
    5:'grocery gourmet food' 
}

# making sure the dataframes are ready for training
def df_process_data(row):
    if isinstance(row['title'], float):
        row['title'] = random.choice(list(cat2idx.keys()))
        return row
    row['title'] = process(row['title'])
    return row

def df_process_labels(row):
    row['category'] = process(row['category'])
    # map it to an integer
    row['category'] = cat2idx[row['category']]
    return row

# process the fields
df_train = df_train.apply(df_process_data, axis=1)
# process the labels
df_train = df_train.apply(df_process_labels, axis=1)
# process the data is the test split
df_test = df_test.apply(df_process_data, axis=1)

# Embeddings

In [10]:
# in the rest of the code I will be using the d
import torch
from transformers import AutoModel, AutoTokenizer
NOTEBOOK_DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
CHECKPOINT = 'distilbert-base-uncased' # let's keep it simple as for the first iteration
MODEL = AutoModel.from_pretrained(CHECKPOINT).to(NOTEBOOK_DEVICE)
TOKENIZER = AutoTokenizer.from_pretrained(CHECKPOINT)

In [19]:
import nlpaug.augmenter.word as naw
import random
random.seed(69)
AUGMENTER1 = naw.ContextualWordEmbsAug(model_path=CHECKPOINT, aug_p=0.1, device=NOTEBOOK_DEVICE)
AUGMENTER2 = naw.ContextualWordEmbsAug(model_path=CHECKPOINT, aug_p=0.1, device=NOTEBOOK_DEVICE)

def augmented_df(df) -> pd.DataFrame:
    aug_d1 = AUGMENTER1.augment(df['title'].tolist())
    aug_d2 = AUGMENTER2.augment(df['title'].tolist())
    df1 = pd.DataFrame(data={"title": aug_d1, "category": df['category']})
    df2 = pd.DataFrame(data={"title": aug_d2, "category": df['category']})
    return pd.concat([df1, df2])

In [22]:
df_train_aug = augmented_df(df_train)

In [23]:
df_train = pd.concat([df_train_aug, df_train])

In [24]:
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(df_train, test_size=0.15, stratify=df_train['category'], random_state=69)

# Train Loaders

In [25]:
from torch.utils.data import DataLoader, Dataset


def collate_function(batch: List[str], aug_prob: float = None):
    # batch will represent a list of tuples (text, category) 
    x, y = [list(row) for row in zip(*batch)]
    # convert both labels and data to tensors    
    y_tensor = torch.FloatTensor(y).to(device=NOTEBOOK_DEVICE)
    embeddings = MODEL(**TOKENIZER(x, padding=True, return_tensors='pt').to(NOTEBOOK_DEVICE)).last_hidden_state # make sure to return tensors
    return embeddings.to(NOTEBOOK_DEVICE), y_tensor

# let's create a dataset object really quick:
class LabeledReviewDS(Dataset):
    def __init__(self, data: pd.DataFrame) -> None:
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index) -> tuple[str, int]:
        return tuple(self.data.iloc[index, :2])

# let's set the random seed

torch.manual_seed(69)

train_ds = LabeledReviewDS(train_data)
val_ds = LabeledReviewDS(val_data)

# create the dataloaders
train_dl = DataLoader(dataset=train_ds, batch_size=64, shuffle=True, collate_fn=lambda x: collate_function(x, aug_prob=0.15), drop_last=True)
val_dl = DataLoader(dataset=val_ds, batch_size=64, shuffle=False, collate_fn=collate_function)

# Train A model

In [26]:
from torch import nn
class SeqClassModel(nn.Module):
    def __init__(self, 
                in_features: int,
                hidden_size: int, 
                num_classes: int, 
                num_layers: int = 2, 
                dropout: float=0.3, 
                *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.output_units = num_classes if num_classes > 2 else 1
        self.rnn = nn.LSTM(input_size=in_features, 
                           hidden_size=hidden_size, 
                           dropout=dropout, 
                           num_layers=num_layers,
                           bidirectional=True, # bidiretional RNN are more powerful
                           batch_first=True # easier manipulation
                           )
        # 2: comes from the fact that the lstm is bidirectional, the rest is similar to the LSTM documention Pytorch
        linear_input_dim = 2 * num_layers * hidden_size 
        self.batch_layer= nn.BatchNorm1d(num_features=linear_input_dim)
        # self.relu_layer = nn.LeakyReLU()
        self.head = nn.Linear(in_features=linear_input_dim, out_features=self.output_units)
        
    def forward(self, x: torch.Tensor):
        # first pass it through the rnn
        _, (hidden_state, _) = self.rnn(x)
        batch_size = hidden_state.shape[1]
        # first permuting channels: batch_size as dimensions '0' 
        # only only the last lstm layer
        hidden_state = hidden_state.permute((1, 0, 2)).reshape((batch_size, -1))
        return self.head.forward(self.batch_layer(hidden_state))


In [31]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import LinearLR
from torchmetrics.classification import MulticlassF1Score, MulticlassAccuracy

base_model = SeqClassModel(in_features=768, hidden_size=128, num_classes=6)
optimizer = AdamW(base_model.parameters(), lr=0.01)
scheduler = LinearLR(optimizer, start_factor=1.0, end_factor=0.005, total_iters=100)

accuracy_metric, f1_metric = MulticlassAccuracy(num_classes=6), MulticlassF1Score(num_classes=6)

metrics = {'accuracy': accuracy_metric, 'f1_score': f1_metric}


train_configuration = {'optimizer': optimizer,
                        'scheduler': scheduler,
                        'min_val_loss': 10 ** -4,
                        'max_epochs': 60,
                        'report_epoch': 1,
                        'device': NOTEBOOK_DEVICE, 
                        'metrics': metrics,
                        'no_improve_stop': 15
                        }

In [32]:
import src.pytorch_modular.image_classification.engine_classification as cls
results = cls.train_model(base_model, train_dl, val_dl, train_configuration,    
                            log_dir=os.path.join(HOME, 'runs'),         
                            save_path=os.path.join(HOME, 'saved_models'))   

[INFO] Created SummaryWriter, saving to: /home/ayhem18/DEV/My_Kaggle_Repo/amazon_reviews/runs/experience_27...


  0%|          | 0/60 [00:00<?, ?it/s]

  2%|▏         | 1/60 [02:43<2:41:10, 163.90s/it]

#########################
training loss: 0.624814041431266
train_accuracy: 0.7833075523376465
train_f1_score: 0.7734293341636658
validation loss : 0.5441950296467923
val_accuracy: 0.8055578470230103
val_f1_score: 0.805313229560852
#########################


  3%|▎         | 2/60 [05:28<2:38:38, 164.12s/it]

...

#########################
training loss: 0.10455742382432164
train_accuracy: 0.9658377170562744
train_f1_score: 0.9611656665802002
validation loss : 0.2779472472925558
val_accuracy: 0.9126416444778442
val_f1_score: 0.9091469645500183
#########################


 95%|█████████▌| 57/60 [2:39:28<12:13, 244.46s/it]

#########################
training loss: 0.10096696730906568
train_accuracy: 0.9676755666732788
train_f1_score: 0.9628610610961914
validation loss : 0.2589647296406593
val_accuracy: 0.9250345230102539
val_f1_score: 0.9185603260993958
#########################


 95%|█████████▌| 57/60 [2:45:42<08:43, 174.42s/it]


RuntimeError: unique_by_key: failed to synchronize: cudaErrorLaunchFailure: unspecified launch failure

# Inference 

In [ ]:
# let's make the damn submission
from src.pytorch_modular.pytorch_utilities import load_model
# base_model = SeqClassModel(in_features=768, hidden_size=128, num_classes=6)
# base_model = load_model(base_model=base_model, path=os.path.join(HOME, 'saved_models', '9-17-15-10.pt'))
# let's create a dataset object really quick:
class TestReviewDS(Dataset):
    def __init__(self, data: pd.DataFrame) -> None:
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index) -> tuple[str, int]:
        return self.data.iloc[index, 1]

# we need a different callate_function
def test_collate_function(batch):
    embeddings = MODEL(**TOKENIZER(batch, padding=True, return_tensors='pt').to(NOTEBOOK_DEVICE)).last_hidden_state # make sure to return tensors
    return embeddings.to(NOTEBOOK_DEVICE)
    
# let's set the random seed

torch.manual_seed(69)

test_ds = TestReviewDS(data=df_test)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, collate_fn=test_collate_function)
# next(iter(test_loader)).shape
predictions = cls.inference(base_model, inference_source_data=test_loader, return_tensor='list')
# convert the numerical labels to the string ones
predictions = [idx2cat[p] for p in predictions]

In [ ]:
submission = pd.DataFrame(data={"id": df_test['id'].tolist(), "Category": predictions})
sub_dir = os.path.join(HOME, 'submissions')
submission.to_csv(os.path.join(sub_dir, f'sub_{len(os.listdir(sub_dir)) + 1}.csv'), index=False)